# Introduction

*General tip 1*: Whenever you modify anything in this notebook, do not forget to click on the save button. In this way, all changes will be saved and you will be able to continue working on your project later on.

*General tip 2*: To run a part of this notebook, you should first click on a section/cell (after selecting it, you should see a blue bar on the left side) and then click the Play button above (alternatively, press *Shift+Enter*). After running a cell, the next one will be automatically selected.

## Initial imports
The following code-cell loads the packages that will be used by the notebook:

In [8]:
%load_ext autoreload
%autoreload 2

from segmfriends.io.images import read_uint8_img
import napari
from magicgui import magicgui
from napari.types import ImageData, LabelsData, ShapesData
from napari.layers import Shapes


import numpy as np
import os
import ipywidgets as widgets
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display
from IPython.display import Markdown, Latex
import imageio

from annotationtools.base_experiment import BaseAnnotationExperiment
from annotationtools.io import file_dialog, dir_dialog
from annotationtools.notebook_utils.widgets_utils import SelectImagePath, display_proj_dir_browse_button

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Select a project directory

<img src="./images/run-cells-in-section.jpg" alt="run-cells-in-section" width="50%" style="float:right; margin-left: 10%;"/>


**Important**: 

-  If not done already, please set the *project_directory* variable in code-cell below.
- **Make sure to run all the cells in this section** every time you run the notebook. To do so, you can also right-click on the Table–of-Contents list on the left side (see screenshot on the side).

**FAQ**:
- **What is the project directory?** The project directory is the folder that will contain all the data and images generated by this notebook. If you are starting a new labeling project, then create and select an empty folder: all generated annotations will be saved in that folder. Otherwise, select the path to a project you have been previously worked on.
- **How do I set the project path?** You can either enter it manually or you can use the "Browse..." button below to get the path of any directory on your machine. After that, do not forget to copy the obtained path in the code cell below where the *project_directory* variable is defined.

In [9]:
# Display "Browse" button to select project directory:
display_proj_dir_browse_button()

In [10]:
# --------------------  TO BE FILLED BY USER - Start  -----------------------------
# Insert path to project directory:
project_directory = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_project" 
# ---------------------  TO BE FILLED BY USER - End  ------------------------------

# Load the annotation project:
annotation_exp = BaseAnnotationExperiment(project_directory)

## Images in your project
By running the code cell below you will see the list of images that are already part of your project.

In [11]:
# Show images in project:

input_images_with_nb_rois = annotation_exp.get_list_rois_per_image()
echo = f"""**Name loaded project:** '*{os.path.split(project_directory)[1]}*' 

"""
nb_images = len(input_images_with_nb_rois)
if nb_images:
    echo += f"""There are {nb_images} images loaded in the project:
    """
else:
    echo += f"""There are no images loaded in the project.
    """


for i, in_image in enumerate(input_images_with_nb_rois):
    echo += f"""
- Image {i+1}:
    - Path main image: *{in_image[0]}*
    - Number of selected regions of interest in image: *{in_image[1]}*  
    """
    
display(Markdown(echo))

**Name loaded project:** '*test_project*' 

There are 1 images loaded in the project:
    
- Image 1:
    - Path main image: */Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_images/img1/fused_tp_0_ch_4.tif*
    - Number of selected regions of interest in image: *3*  
    

# Add a new image/dataset to the project
**Important:** If the image you want is already in the project, you don't need to fill the fields shown in this section and you can directly move on to the next section.


In [12]:
from annotationtools.notebook_utils.add_proj_image import AddNewImageToProject
add_new_image = AddNewImageToProject(annotation_exp)
add_new_image.display_notebook_widgets()


Use the fields below to add an additional image to the project:
- **Main channel**: select the path of the main channel that should be segmented (usually the bright-field channel)
- **DAPI channel**: If you have it, please provide it because it can be very useful for getting a better segmentation
- **Additional channels**: If you have additional channels that may helpful for the manual annotation process, you can load two extra ones. 
- **How to specify image paths:**
    - *Option 1*: Specify all channel paths manually
    - *Option 2*: If all channel images are in the same directory and have similar names, e.g. *img_ch0.tif*, *img_ch1.tif*, *img_ch2.tif*, 
you only need to provide the filename filters (*_ch0*, *_ch1*, *_ch2*, ...) and the paths of the additional channels
will be automatically deduced from the main-channel image.
- When you have specified all the paths, press on the submit button *"Add image to project"*. 


# Select regions of interest to be annotated
In this section, you will load an image and then select one or more region of interests (rectangular boxes) that you will manually annotate in the following and will be used to train the segmentation algorithm. 

## Things to keep in mind when selecting regions of interest
- When you select a region of interest, keep in mind that **you will need to manually annotate all cells inside that region of interest**. Each region of interest should contain **at least several dozens of cells, ideally ~100**.
- These region of interests will be used to train the segmentation algorithm, so you should select the most representative parts of your data. If there are some particularly challenging parts of your data the you would like the segmentation algorithm to properly segment, then you should also select some regions of interest from these difficult parts. 

Here you have some example scenarios:
1. Your images/datasets present several artifacts close to the borders, but you only care to have an accurate segmentation in the central part of the image, where image quality is higher and there are no artifacts. In this case, you can select regions of interest only from the central area of your image.
2. Your images/datasets present several artifacts in all parts of the image, and you would like cells to be properly segmented even in parts of the image where these artifacts are present. Then, you should also include regions of interest where some of the artifacts are visible (on top of region of interests that are easier to segment)
3. You acquired several images that look somehow different (different acquisition method, brightness, etc). Then, you should evenly select regions of interest across all your images/datasets.

## Select the image you want to work with

In [6]:
input_images_with_nb_rois = annotation_exp.get_list_rois_per_image()

dropdown_widget = widgets.Dropdown(
    options=[(img_data[0], i)  for i, img_data in enumerate(input_images_with_nb_rois)],
    value=0,
    disabled=False,
    layout=widgets.Layout(width='90%')
)

id_selected_image = 0

def on_dropdown_value_change(change):
    global id_selected_image
    id_selected_image = change['new']

dropdown_widget.observe(on_dropdown_value_change, names='value')

image_choice_widgets = widgets.VBox([widgets.Label(value="Select an image in the project: "), dropdown_widget])
display(image_choice_widgets)

## Modify / create regions of interest in Napari
After you run the next cell, the selected image will be loaded in Napari. 

Then select the "Rectangle" tool in napari to draw one or more region of interests.

After you are done, close the Napari window and run the  the next sections 

- Select, move, and delete boxes
- Insert image of Napari viewer? (Showing the various buttons)

<!-- ![Napari](./napari-screenshot.jpg) -->

Run the following cell to start napari:

In [7]:
from annotationtools.notebook_utils.create_rois import CreateROIs
create_roi = CreateROIs(annotation_exp)
create_roi.display_widgets()

In [18]:
assert id_selected_image is not None

image_paths = annotation_exp.get_image_paths(id_selected_image)

# create the viewer and display the image
viewer = napari.Viewer()

channel_colormaps = ["gray", "red", "yellow", "cyan"]
for i, channel in enumerate(image_paths):
    viewer.add_image(read_uint8_img(image_paths[channel])[...,0], name=channel, colormap=channel_colormaps[i],
                    blending='additive')

napari_rois = annotation_exp.get_napari_roi_by_image_id(id_selected_image)
    
# Load images in napari:
s_layer = viewer.add_shapes(data=napari_rois, name="ROIs image {}".format(id_selected_image), shape_type="rectangle", opacity=0.15,  edge_color='#fff01dff', face_color='#f9ffbeff')


@magicgui(
    call_button="Save regions of interest",
    shapes={'label': 'ROIs layer:'
           },
)
def save_ROIs(shapes: Shapes): 
    id_image = int(shapes.name.split(" ")[2])
    print(id_image)
    if len(shapes.data):
        annotation_exp.update_rois_image(id_image, shapes.data)

viewer.window.add_dock_widget(save_ROIs, area='right')  # Add our gui instance to napari viewer


ROIs image 0


Run the following cell when you are done selecting the ROIs in napari:

In [12]:
# TODO: add event to napari and call this every time a ROI is updated:
# print(napari_rois.shape)

ROIs_data = s_layer.data
if len(ROIs_data):
    annotation_exp.update_rois_image(id_selected_image, ROIs_data)
else:
    print("No ROIs found for selected image")

# Manual annotation


In [30]:
rois_list = annotation_exp.get_roi_list()

dropdown_image_to_annotate = widgets.Dropdown(
    options=[("Image {} - Region of interest {} - {}".format(roi_info['image_id']+1, roi_info['roi_index_per_image']+1,
                                                          "(has labels)" if roi_info['has_label'] else "(DOES NOT HAVE LABELS)"), 
              roi_info['roi_id'])  
             for roi_info in rois_list],
    disabled=False,
    layout=widgets.Layout(width='90%')
)

id_roi_to_annotate = rois_list[0]['roi_id']

def on_image_to_annotate_value_change(change):
    global id_roi_to_annotate
    id_roi_to_annotate = change['new']

dropdown_image_to_annotate.observe(on_image_to_annotate_value_change, names='value')

image_choice_widgets = widgets.VBox([widgets.Label(value="Select ROI to annotate: "), dropdown_image_to_annotate])
display(image_choice_widgets)

In [37]:
assert id_roi_to_annotate is not None

print(id_roi_to_annotate)
roi_paths = annotation_exp.get_training_image_paths(id_roi_to_annotate)

# create the viewer and display the image
viewer = napari.Viewer()

composite_image = read_uint8_img(roi_paths["composite_image"])

# composite_image = img = imageio.imread(roi_paths["composite_image"])
# print(roi_paths["composite_image"], composite_image.shape)
# assert np.allclose(composite_image[...,0], composite_image[...,1])
# for channel in reversed(roi_paths["single_channels"]):
#     viewer.add_image(read_uint8_img(roi_paths["single_channels"][channel]), name=channel)
viewer.add_image(composite_image, 
                 name=[channel for channel in roi_paths["single_channels"]], 
                 colormap = ["gray", "red", "yellow", "cyan"],
                 channel_axis=2)
    

annotations = imageio.imread(roi_paths["label_image"]) if roi_paths["has_labels"] else np.zeros(shape=composite_image.shape[:2], dtype='uint16')
    
# Load images in napari:
labels_layer = viewer.add_labels(annotations, name='Annotations')

2
uint8


In [38]:
annotation_exp.update_roi_labels(id_roi_to_annotate, labels_layer.data.astype('uint16'))

# Training
TODO: copy actual labels left in project in another training folder (some may have been deleted)